<a href="https://colab.research.google.com/github/ACTP2002/EVIDENCE/blob/behavior_model/Test%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import joblib
import pandas as pd
import numpy as np
from scipy.stats import median_abs_deviation

artifact = joblib.load("behavior_iforest.pkl")
model = artifact["model"]
threshold = artifact["threshold"]

test_df = pd.read_csv("test_transactions 2.csv")
test_df["event_time"] = pd.to_datetime(test_df["event_time"])

test_df["amount_abs"] = test_df["amount"].abs()

grp = test_df.groupby("user_id", sort=False)

# Rolling windows (15 transactions minimum, 7 transactions at least to compute)
test_df["amt_roll_med_15"] = grp["amount_abs"].transform(lambda s: s.rolling(15, min_periods=7).median())

# Rolling MAD
test_df["amt_roll_mad_15"] = grp["amount_abs"].transform(lambda s: s.rolling(15, min_periods=7).apply(
    lambda x: median_abs_deviation(x, scale="normal"), raw=False
))

test_df["amt_dev_from_med"] = test_df["amount_abs"] - test_df["amt_roll_med_15"]
test_df["amt_robust_z"] = test_df["amt_dev_from_med"] / (test_df["amt_roll_mad_15"] + 1e-9)

test_df["prev_event_time"] = grp["event_time"].shift(1)
# Gap between consecutive transactions (in seconds)
test_df["gap_seconds"] = (test_df["event_time"] - test_df["prev_event_time"]).dt.total_seconds()

# Fill first-transaction gaps
test_df["gap_seconds"] = test_df["gap_seconds"].fillna(test_df["gap_seconds"].median())

# Log-scaled time gap
test_df["gap_log"] = np.log1p(test_df["gap_seconds"])

test_df["deposit_to_income_ratio"] = (test_df["account_deposit"] / (test_df["declared_income"] + 1e-9))

test_df["amount_to_income_ratio"] = (test_df["amount_abs"] / (test_df["declared_income"] + 1e-9))

test_df["net_flow_1d"] = test_df["amount_in_1d"] - test_df["amount_out_1d"]

test_df["failed_login_ratio_1h"] = (test_df["failed_login_1h"] / (test_df["login_count_1h"] + 1e-9))
test_df["new_ip_1d"] = test_df["new_ip_1d"].fillna(0)
test_df["geo_change_1d"] = test_df["geo_change_1d"].fillna(0)

test_df["is_cross_border"] = (test_df["residence_country"] != test_df["transaction_country"]).astype(int)

raw = model.decision_function(test_df)
test_df["anomaly_score"] = -raw
test_df["is_anomaly"] = (test_df["anomaly_score"] >= threshold).astype(int)

print(
    test_df[[
        "user_id", "event_time", "event_type",
        "amount", "transaction_country",
        "failed_login_1h", "new_ip_1d",
        "anomaly_score", "is_anomaly"
    ]]
)


/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid r

   user_id          event_time  event_type  amount transaction_country  \
0    U1001 2026-03-01 09:00:00     deposit     500                  us   
1    U1001 2026-03-01 12:00:00         buy     200                  us   
2    U1001 2026-03-01 18:00:00  withdrawal     300                  us   
3    U1001 2026-03-02 09:30:00         buy     150                  us   
4    U2001 2026-03-01 10:00:00     deposit     800                  us   
5    U2001 2026-03-01 15:00:00        sell     300                  us   
6    U2001 2026-03-02 11:00:00  withdrawal     500                  us   
7    U9001 2026-03-03 01:10:00     deposit    1000                  us   
8    U9001 2026-03-03 01:12:00  withdrawal   25000                  sg   
9    U9001 2026-03-03 01:13:00  withdrawal   18000                  sg   
10   U9002 2026-03-04 22:00:00     deposit     600                  us   
11   U9002 2026-03-04 22:01:00  withdrawal   15000                  ru   
12   U3001 2026-03-05 09:00:00     dep